In [10]:
from pyotp import TOTP
import os
import time
from kiteconnect import KiteConnect 
from selenium import webdriver
import pandas as pd
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager



token_path="access_key.rtf"
api_secrets=open(token_path,'r').read().split()
api_key=api_secrets[-3].replace('\\','')
api_secret=api_secrets[-2].replace('\\','')
user_name=api_secrets[-5].replace('\\','')
password=api_secrets[-4].replace('\\','')
totp=api_secrets[-6].replace('\\','')
kite=KiteConnect(api_key)

def auto_login():
    # Set up the service with the ChromeDriverManager
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(kite.login_url())
    driver.implicitly_wait(10)
    username=driver.find_element('xpath','/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[1]/input')
    password_f=driver.find_element('xpath','/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[2]/input')
    username.send_keys(user_name)
    password_f.send_keys(password)
    driver.find_element('xpath','/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[4]/button').click()
    time.sleep(5)
    pin=driver.find_element('xpath','/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[1]/input')
    token_t=TOTP(totp)
    code=token_t.now()
    pin.send_keys(code)
    time.sleep(10)
    request_token=driver.current_url.split("request_token=")[-1]
    with open('request_token.txt','w') as f:
        f.write(request_token)
    driver.quit()

In [11]:
auto_login()
request_tokens=open('request_token.txt','r').read().split('&')[0]
data = kite.generate_session(request_tokens,api_secret)


In [12]:
instrument_dump=kite.instruments("NSE")
instrument_df=pd.DataFrame(instrument_dump)

In [13]:
instrument_df

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.00,0,EQ,INDICES,NSE
1,256777,1003,NIFTY MIDCAP 100,NIFTY MIDCAP 100,0.0,,0.0,0.00,0,EQ,INDICES,NSE
2,260105,1016,NIFTY BANK,NIFTY BANK,0.0,,0.0,0.00,0,EQ,INDICES,NSE
3,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.00,0,EQ,INDICES,NSE
4,257033,1004,NIFTY DIV OPPS 50,NIFTY DIV OPPS 50,0.0,,0.0,0.00,0,EQ,INDICES,NSE
...,...,...,...,...,...,...,...,...,...,...,...,...
7261,8150273,31837,SREEL,SREELEATHERS,0.0,,0.0,0.05,1,EQ,NSE,NSE
7262,8183297,31966,863NHB29-N2,,0.0,,0.0,0.01,1,EQ,NSE,NSE
7263,8183553,31967,863NHB34-N3,,0.0,,0.0,0.01,1,EQ,NSE,NSE
7264,8184065,31969,888NHB29-N5,,0.0,,0.0,0.01,1,EQ,NSE,NSE


In [14]:
nifty_bank=["HDFC Bank","ICICI Bank","Axis Bank","State Bank of India","Kotak Mahindra Bank","IndusInd Bank",
            "Bank of Baroda","Federal Bank","Punjab National Bank","Canara Bank","AU Small Finance Bank","IDFC First Bank"]

In [15]:
import pandas as pd

bank_tradingsymbol_list = []  
bank_instrument_tokens_list = [] 
bank_companies_with_symbols = []  
for company in nifty_bank:  
    
    full_name = company.strip() 

   
    full_match = instrument_df[instrument_df['name'].str.contains(full_name, case=False, na=False)]
    
    if len(full_match) == 1: 
        bank_tradingsymbol_list.append(full_match['tradingsymbol'].iloc[0])
        bank_instrument_tokens_list.append(full_match['instrument_token'].iloc[0])
        bank_companies_with_symbols.append(company)
    else: 
        bank_tradingsymbol_list.append(None)
        bank_instrument_tokens_list.append(None)
        bank_companies_with_symbols.append(company)


bank_result_df = pd.DataFrame({
    'Company': bank_companies_with_symbols,
    'Instrument Token': bank_instrument_tokens_list,
    'Trading Symbol': bank_tradingsymbol_list
})




In [16]:
bank_result_df

,Company,Instrument Token,Trading Symbol
0,HDFC Bank,341249,HDFCBANK
1,ICICI Bank,1270529,ICICIBANK
2,Axis Bank,1510401,AXISBANK
3,State Bank of India,779521,SBIN
4,Kotak Mahindra Bank,492033,KOTAKBANK
5,IndusInd Bank,1346049,INDUSINDBK
6,Bank of Baroda,1195009,BANKBARODA
7,Federal Bank,261889,FEDERALBNK
8,Punjab National Bank,2730497,PNB
9,Canara Bank,2763265,CANBK


In [17]:
len(bank_tradingsymbol_list)

12

In [18]:
import pandas as pd
from datetime import datetime, timedelta
from io import StringIO
import boto3

aws_access_key = '---------'
aws_secret_key = '----------------'
bucket_name = 'planwo-stockmarket-macd'

session = boto3.Session(
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key
)
s3 = session.client('s3')

end_date = datetime.now() - timedelta(days=1)
start_date = end_date - timedelta(days=15*365)  
intervals = {
    "15minute": 30,
    "30minute": 200,
    "60minute": 400
}

def fetch_historical_data_in_chunks(instrument_token, start_date, end_date, interval, max_days):
    data = []
    current_start_date = start_date
    while current_start_date < end_date:
        current_end_date = min(current_start_date + timedelta(days=max_days), end_date)
        try:
            chunk_data = kite.historical_data(
                instrument_token,
                current_start_date.strftime("%Y-%m-%d"),
                current_end_date.strftime("%Y-%m-%d"),
                interval
            )
            if chunk_data:
                data.extend(chunk_data)
            current_start_date = current_end_date + timedelta(days=1)
        except Exception as e:
            print(f"Error fetching data from {current_start_date} to {current_end_date}: {e}")
            break
    return data

def calculate_rsi(data, window=14):
    delta = data['close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_macd(data, short_window=12, long_window=26):
    short_ema = data['close'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    return macd

def aggregate_data(df, interval):
    if interval == "day":
        df['Date'] = pd.to_datetime(df['date']).dt.date
        agg_data = df.groupby('Date').agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        }).reset_index()
    elif interval == "week":
        df['Date'] = pd.to_datetime(df['date']).dt.to_period('W').dt.start_time
        agg_data = df.groupby('Date').agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        }).reset_index()
    elif interval == "month":
        df['Date'] = pd.to_datetime(df['date']).dt.to_period('M').dt.start_time
        agg_data = df.groupby('Date').agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        }).reset_index()
    return agg_data


for company, trading_symbol, instrument_token in zip(bank_companies_with_symbols, bank_tradingsymbol_list, bank_instrument_tokens_list):
    if not trading_symbol or not instrument_token:
        print(f"Skipping company {company} due to missing data.")
        continue

    historical_data_list = []
    for interval, max_days in intervals.items():
        print(f"Fetching {interval} data for {company}...")
        data = fetch_historical_data_in_chunks(instrument_token, start_date, end_date, interval, max_days)
        if data:
            historical_data_df = pd.DataFrame(data)
            historical_data_df['Company'] = company
            historical_data_df['Trading Symbol'] = trading_symbol
            historical_data_df['Interval'] = interval
            historical_data_list.append(historical_data_df)

    if not historical_data_list:
        print(f"No data fetched for {company}.")
        continue

    
    all_historical_data = pd.concat(historical_data_list, ignore_index=True)

    
    day_data = aggregate_data(all_historical_data, "day")
    week_data = aggregate_data(all_historical_data, "week")
    month_data = aggregate_data(all_historical_data, "month")

    interval_data_dict = {
        "15minute": [df for df in historical_data_list if df['Interval'].iloc[0] == "15minute"][0][['date', 'open', 'high', 'low', 'close', 'volume']],
        "30minute": [df for df in historical_data_list if df['Interval'].iloc[0] == "30minute"][0][['date', 'open', 'high', 'low', 'close', 'volume']],
        "60minute": [df for df in historical_data_list if df['Interval'].iloc[0] == "60minute"][0][['date', 'open', 'high', 'low', 'close', 'volume']],
        "1day": day_data.rename(columns={"Date": "date"})[['date', 'open', 'high', 'low', 'close', 'volume']],
        "1week": week_data.rename(columns={"Date": "date"})[['date', 'open', 'high', 'low', 'close', 'volume']],
        "1month": month_data.rename(columns={"Date": "date"})[['date', 'open', 'high', 'low', 'close', 'volume']]
    }

    for interval, df in interval_data_dict.items():
        
        df = df.sort_values(by='date').reset_index(drop=True)

      
        df['SMA_50'] = df['close'].rolling(window=50).mean()  # Simple Moving Average for 50 periods
        df['EMA_50'] = df['close'].ewm(span=50, adjust=False).mean()  # Exponential Moving Average for 50 periods

        # Calculate RSI
        df['RSI'] = calculate_rsi(df, window=14)

        # Calculate MACD (12-day EMA - 26-day EMA)
        df['MACD'] = calculate_macd(df)

        csv_bfr = StringIO()
        df.to_csv(csv_bfr, index=False)
        csv_bfr.seek(0)
        object_key = f"Historical data/BankNifty/{company}/{interval}.csv"
        s3.put_object(Bucket=bucket_name, Key=object_key, Body=csv_bfr.getvalue())
        print(f"{interval} data with SMA, EMA, RSI, and MACD uploaded successfully for {company} to s3://{bucket_name}/{object_key}")


Fetching 15minute data for HDFC Bank...
Fetching 30minute data for HDFC Bank...
Fetching 60minute data for HDFC Bank...


C:\Users\Alisia Phini\AppData\Local\Temp\ipykernel_9276\2023383984.py:73: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['Date'] = pd.to_datetime(df['date']).dt.to_period('W').dt.start_time
C:\Users\Alisia Phini\AppData\Local\Temp\ipykernel_9276\2023383984.py:82: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['Date'] = pd.to_datetime(df['date']).dt.to_period('M').dt.start_time


15minute data with SMA, EMA, RSI, and MACD uploaded successfully for HDFC Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/HDFC Bank/15minute.csv
30minute data with SMA, EMA, RSI, and MACD uploaded successfully for HDFC Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/HDFC Bank/30minute.csv
60minute data with SMA, EMA, RSI, and MACD uploaded successfully for HDFC Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/HDFC Bank/60minute.csv
1day data with SMA, EMA, RSI, and MACD uploaded successfully for HDFC Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/HDFC Bank/1day.csv
1week data with SMA, EMA, RSI, and MACD uploaded successfully for HDFC Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/HDFC Bank/1week.csv
1month data with SMA, EMA, RSI, and MACD uploaded successfully for HDFC Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/HDFC Bank/1month.csv
Fetching 15minute data for ICICI Bank...
Fetching 30minute

C:\Users\Alisia Phini\AppData\Local\Temp\ipykernel_9276\2023383984.py:73: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['Date'] = pd.to_datetime(df['date']).dt.to_period('W').dt.start_time
C:\Users\Alisia Phini\AppData\Local\Temp\ipykernel_9276\2023383984.py:82: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['Date'] = pd.to_datetime(df['date']).dt.to_period('M').dt.start_time


15minute data with SMA, EMA, RSI, and MACD uploaded successfully for ICICI Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/ICICI Bank/15minute.csv
30minute data with SMA, EMA, RSI, and MACD uploaded successfully for ICICI Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/ICICI Bank/30minute.csv
60minute data with SMA, EMA, RSI, and MACD uploaded successfully for ICICI Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/ICICI Bank/60minute.csv
1day data with SMA, EMA, RSI, and MACD uploaded successfully for ICICI Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/ICICI Bank/1day.csv
1week data with SMA, EMA, RSI, and MACD uploaded successfully for ICICI Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/ICICI Bank/1week.csv
1month data with SMA, EMA, RSI, and MACD uploaded successfully for ICICI Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/ICICI Bank/1month.csv
Fetching 15minute data for Axis Bank...
Fetchi

...

15minute data with SMA, EMA, RSI, and MACD uploaded successfully for Canara Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/Canara Bank/15minute.csv
30minute data with SMA, EMA, RSI, and MACD uploaded successfully for Canara Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/Canara Bank/30minute.csv
60minute data with SMA, EMA, RSI, and MACD uploaded successfully for Canara Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/Canara Bank/60minute.csv
1day data with SMA, EMA, RSI, and MACD uploaded successfully for Canara Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/Canara Bank/1day.csv
1week data with SMA, EMA, RSI, and MACD uploaded successfully for Canara Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/Canara Bank/1week.csv
1month data with SMA, EMA, RSI, and MACD uploaded successfully for Canara Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/Canara Bank/1month.csv
Fetching 15minute data for AU Smal

C:\Users\Alisia Phini\AppData\Local\Temp\ipykernel_9276\2023383984.py:73: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['Date'] = pd.to_datetime(df['date']).dt.to_period('W').dt.start_time
C:\Users\Alisia Phini\AppData\Local\Temp\ipykernel_9276\2023383984.py:82: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['Date'] = pd.to_datetime(df['date']).dt.to_period('M').dt.start_time


15minute data with SMA, EMA, RSI, and MACD uploaded successfully for AU Small Finance Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/AU Small Finance Bank/15minute.csv
30minute data with SMA, EMA, RSI, and MACD uploaded successfully for AU Small Finance Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/AU Small Finance Bank/30minute.csv
60minute data with SMA, EMA, RSI, and MACD uploaded successfully for AU Small Finance Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/AU Small Finance Bank/60minute.csv
1day data with SMA, EMA, RSI, and MACD uploaded successfully for AU Small Finance Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/AU Small Finance Bank/1day.csv
1week data with SMA, EMA, RSI, and MACD uploaded successfully for AU Small Finance Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/AU Small Finance Bank/1week.csv
1month data with SMA, EMA, RSI, and MACD uploaded successfully for AU Small Finance Bank to s3

C:\Users\Alisia Phini\AppData\Local\Temp\ipykernel_9276\2023383984.py:73: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['Date'] = pd.to_datetime(df['date']).dt.to_period('W').dt.start_time
C:\Users\Alisia Phini\AppData\Local\Temp\ipykernel_9276\2023383984.py:82: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['Date'] = pd.to_datetime(df['date']).dt.to_period('M').dt.start_time


15minute data with SMA, EMA, RSI, and MACD uploaded successfully for IDFC First Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/IDFC First Bank/15minute.csv
30minute data with SMA, EMA, RSI, and MACD uploaded successfully for IDFC First Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/IDFC First Bank/30minute.csv
60minute data with SMA, EMA, RSI, and MACD uploaded successfully for IDFC First Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/IDFC First Bank/60minute.csv
1day data with SMA, EMA, RSI, and MACD uploaded successfully for IDFC First Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/IDFC First Bank/1day.csv
1week data with SMA, EMA, RSI, and MACD uploaded successfully for IDFC First Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/IDFC First Bank/1week.csv
1month data with SMA, EMA, RSI, and MACD uploaded successfully for IDFC First Bank to s3://planwo-stockmarket-macd/Historical data/BankNifty/IDFC First Ba